In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import poisson,skellam
from scipy.optimize import minimize

df = pd.read_csv('ncaa_mens_scores_2024.csv')

In [3]:
df.columns

Index(['home_team', 'home_team_score', 'away_team', 'away_team_score',
       'home_team_conference', 'away_team_conference'],
      dtype='object')

In [9]:
conferences = df['home_team_conference'].unique()

In [23]:
conferences

array(['SoCon', 'Atlantic 10', 'CAA', 'Patriot', 'Big East', 'NEC',
       'Horizon', 'Summit League', 'ACC', 'Sun Belt', 'MAAC', 'MVC',
       'America East', 'OVC', 'The American', 'Big Ten', 'AAC', 'ASUN',
       'Big West', 'WCC', 'WAC', 'Big South', 'Ivy League', 'Not D1'],
      dtype=object)

In [21]:
for conference in conferences:
    # Create a new DataFrame filtered by the conference
    filtered_df = df[(df['home_team_conference'] == conference) & (df['away_team_conference'] == conference)]
    
    # Assign the DataFrame to a variable with a dynamic name
    result_name = f"{conference.replace(' ', '_')}_Results"
    globals()[result_name] = filtered_df


In [69]:
SoCon_teams = SoCon_Results['home_team'].unique()

In [83]:
Patriot_Results

,home_team,home_team_score,away_team,away_team_score,home_team_conference,away_team_conference
589,Bucknell,2,Navy,1,Patriot,Patriot
590,Boston U.,3,Army West Point,2,Patriot,Patriot
598,American,2,Loyola Maryland,0,Patriot,Patriot
608,Colgate,2,Lafayette,1,Patriot,Patriot
612,Holy Cross,2,Lehigh,1,Patriot,Patriot
749,Lafayette,1,Navy,2,Patriot,Patriot
756,Loyola Maryland,2,Boston U.,2,Patriot,Patriot
759,Colgate,1,American,1,Patriot,Patriot
760,Bucknell,0,Holy Cross,0,Patriot,Patriot
782,Army West Point,2,Lehigh,3,Patriot,Patriot


In [71]:
import pandas as pd

# Initialize standings
standings = pd.DataFrame(SoCon_teams, columns=['Team'])
standings['GP'] = 0
standings['Wins'] = 0
standings['Draws'] = 0
standings['Losses'] = 0
standings['Points'] = 0
standings['GF'] = 0  # Goals For
standings['GA'] = 0  # Goals Against

# Process results
for _, game in SoCon_Results.iterrows():
    home_team = game['home_team']
    away_team = game['away_team']
    home_score = game['home_team_score']
    away_score = game['away_team_score']
    
    # Update goals for and against
    standings.loc[standings['Team'] == home_team, 'GF'] += home_score
    standings.loc[standings['Team'] == away_team, 'GF'] += away_score
    standings.loc[standings['Team'] == home_team, 'GA'] += away_score
    standings.loc[standings['Team'] == away_team, 'GA'] += home_score
    
    # Update match results
    if home_score > away_score:
        standings.loc[standings['Team'] == home_team, 'Wins'] += 1
        standings.loc[standings['Team'] == home_team, 'Points'] += 3
        standings.loc[standings['Team'] == away_team, 'Losses'] += 1
    elif home_score < away_score:
        standings.loc[standings['Team'] == away_team, 'Wins'] += 1
        standings.loc[standings['Team'] == away_team, 'Points'] += 3
        standings.loc[standings['Team'] == home_team, 'Losses'] += 1
    else:
        standings.loc[standings['Team'] == home_team, 'Draws'] += 1
        standings.loc[standings['Team'] == away_team, 'Draws'] += 1
        standings.loc[standings['Team'] == home_team, 'Points'] += 1
        standings.loc[standings['Team'] == away_team, 'Points'] += 1

# Calculate games played
standings['GP'] = standings[['Wins', 'Draws', 'Losses']].sum(axis=1)

In [73]:
standings

,Team,GP,Wins,Draws,Losses,Points,GF,GA
0,Mercer,6,1,2,3,5,5,6
1,ETSU,7,3,2,2,11,9,7
2,VMI,6,0,0,6,0,2,15
3,UNC Greensboro,6,3,2,1,11,9,7
4,Wofford,7,4,1,2,13,10,8
5,Furman,8,4,3,1,15,13,5


In [79]:
SoCon_Furman = SoCon_Results[(SoCon_Results['home_team'] == 'Furman') | (SoCon_Results['away_team'] == 'Furman')]

In [81]:
SoCon_Furman

,home_team,home_team_score,away_team,away_team_score,home_team_conference,away_team_conference
1223,UNC Greensboro,1,Furman,0,SoCon,SoCon
1385,Furman,4,VMI,1,SoCon,SoCon
1427,Furman,2,Wofford,0,SoCon,SoCon
1524,ETSU,1,Furman,1,SoCon,SoCon
1689,Furman,1,Mercer,1,SoCon,SoCon
1773,Furman,2,Mercer,0,SoCon,SoCon
1806,UNC Greensboro,1,Furman,1,SoCon,SoCon
1882,ETSU,0,Furman,2,SoCon,SoCon
